# Model
### Wav2Vec2

In [132]:
import pandas as pd

In [9]:
import os, json, time, re, random, numpy as np
from pathlib import Path
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

try:
    import torchaudio
    HAVE_TA = True
except Exception:
    HAVE_TA = False
    print("[WARN] torchaudio not found, augmentation disabled.")

import pyedflib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, f1_score, precision_recall_fscore_support,
    classification_report, confusion_matrix, roc_auc_score,
    log_loss, average_precision_score
)

from transformers import (
    Wav2Vec2Processor, Wav2Vec2ForSequenceClassification,
    get_linear_schedule_with_warmup
)

In [10]:
# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# Prefer CUDA (your RTX 4060). Fallback to CPU if unavailable.
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using device:", device)

Using device: cuda


In [135]:
CSV_PATH = r"C:\V89\Snore_Apnea_Analyze\EDF_RML\data_csv\respiratory_plus_normal.csv"
EDF_ROOT = r"C:\V89\data2"

MODEL_NAME   = "facebook/wav2vec2-base"
MODEL_TAG    = "wav2vec2-base-osa-win4060-v2"  # เปลี่ยนชื่อใหม่

SAMPLE_RATE  = 16000
MAX_SECONDS  = 8      
BATCH_SIZE   = 4
EPOCHS       = 5      # เพิ่ม epochs
LR           = 1e-5   # ลด learning rate
WARMUP_RATIO = 0.1
FREEZE_BASE  = False  # **เปลี่ยนเป็น False - นี่คือปัญหาหลัก**
UNFREEZE_EPOCH = None # ปิดการใช้งาน
AUGMENT      = True        
USE_CLASS_WEIGHTS = True   # เปิดใช้ class weights

In [12]:
class SleepApneaDataset(Dataset):
    """
    Expect df with columns: patient_id, type, segment_index, segment_local_start_sec, duration_sec, ...
    Resolve EDF path once and drop rows without matches.
    """
    def __init__(self, df: pd.DataFrame, edf_root: str, sample_rate: int = 16000,
                 prefer_audio_channels=("sound","snore","tracheal","microphone","audio","throat")):
        self.sample_rate = sample_rate
        self.edf_root = Path(edf_root)
        self.prefer_audio_channels = tuple(s.lower() for s in prefer_audio_channels)

        df = df.copy()
        df["pid_str"]   = df["patient_id"].astype(str)
        df["pid_unpad"] = df["pid_str"].str.lstrip("0")
        df["pid_pad8"]  = df["pid_unpad"].str.zfill(8)
        df["seg3"]      = df["segment_index"].astype(int).map(lambda x: f"{x:03d}")

        def resolve_row(row):
            pid_unpad = row["pid_unpad"]
            pid_pad8  = row["pid_pad8"]
            seg3      = row["seg3"]
            patterns = [
                f"*{pid_pad8}*{seg3}*.edf",
                f"*{pid_unpad}*{seg3}*.edf",
                f"*{pid_pad8}*.edf",
                f"*{pid_unpad}*.edf",
            ]
            for pat in patterns:
                hits = list(self.edf_root.rglob(pat))
                if len(hits) == 1:
                    return hits[0]
                if len(hits) > 1:
                    ranked = sorted(
                        hits,
                        key=lambda p: (
                            0 if re.search(rf"{seg3}\b", p.stem) else 1,
                            0 if re.search(r"(snore|sound|trach|mic|psg|audio|throat)", p.stem.lower()) else 1,
                            len(p.as_posix())
                        )
                    )
                    return ranked[0]
            return None

        df["edf_path"] = df.apply(resolve_row, axis=1)
        missing = df["edf_path"].isna().sum()
        if missing:
            print(f"[WARNING] Skipping {missing} rows that have no matching EDF file.")
        df = df.dropna(subset=["edf_path"]).reset_index(drop=True)

        self.df = df
        self.labels = sorted(self.df['type'].unique().tolist())
        self.label_encoder = LabelEncoder().fit(self.labels)

    def __len__(self):
        return len(self.df)

    def _pick_channel_index(self, f: pyedflib.EdfReader):
        labels = [s.lower() for s in f.getSignalLabels()]
        for i, name in enumerate(labels):
            if any(key in name for key in self.prefer_audio_channels):
                return i
        return 0  # fallback

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        edf_path = Path(row["edf_path"])
        start_sec = float(row['segment_local_start_sec'])
        duration_sec = float(row['duration_sec'])
        label = self.label_encoder.transform([row['type']])[0]

        f = pyedflib.EdfReader(str(edf_path))
        ch_idx = self._pick_channel_index(f)
        signal = f.readSignal(ch_idx)
        fs = f.getSampleFrequency(ch_idx)
        f.close()

        start_sample = max(0, int(start_sec * fs))
        end_sample   = min(int((start_sec + duration_sec) * fs), len(signal))
        audio = signal[start_sample:end_sample]

        x = torch.tensor(audio, dtype=torch.float32)
        if fs != self.sample_rate:
            if not HAVE_TA:
                raise RuntimeError("torchaudio is required for resampling")
            x = torchaudio.transforms.Resample(fs, self.sample_rate)(x)

        return {"audio": x, "label": int(label)}

In [117]:
df = pd.read_csv(CSV_PATH)
dataset = SleepApneaDataset(df, edf_root=EDF_ROOT, sample_rate=SAMPLE_RATE)

print("Resolved EDF rows:", len(dataset))
print(dataset.df[["patient_id","pid_unpad","pid_pad8","segment_index","edf_path"]].head(8))

label_names = list(dataset.label_encoder.classes_)
n_classes = len(label_names)
print("Classes:", label_names, " | n_classes =", n_classes)

[WARNING] Skipping 245 rows that have no matching EDF file.
Resolved EDF rows: 1824
   patient_id pid_unpad  pid_pad8  segment_index  \
0         999       999  00000999              0   
1         999       999  00000999              0   
2         999       999  00000999              0   
3         999       999  00000999              0   
4         999       999  00000999              0   
5         999       999  00000999              0   
6         999       999  00000999              0   
7         999       999  00000999              0   

                                edf_path  
0  C:\V89\data2\00000999-100507[001].edf  
1  C:\V89\data2\00000999-100507[001].edf  
2  C:\V89\data2\00000999-100507[001].edf  
3  C:\V89\data2\00000999-100507[001].edf  
4  C:\V89\data2\00000999-100507[001].edf  
5  C:\V89\data2\00000999-100507[001].edf  
6  C:\V89\data2\00000999-100507[001].edf  
7  C:\V89\data2\00000999-100507[001].edf  
Classes: [np.str_('CentralApnea'), np.str_('Hypopnea'), np.s

In [118]:
if SPLIT_FILE.exists():
    with open(SPLIT_FILE, "r") as f:
        idx = json.load(f)
    train_idx, val_idx, test_idx = idx["train"], idx["val"], idx["test"]
else:
    g = torch.Generator().manual_seed(SEED)
    N = len(dataset)
    perm = torch.randperm(N, generator=g).tolist()
    train_ratio, val_ratio = 0.8, 0.1
    n_train = int(train_ratio * N)
    n_val   = int(val_ratio   * N)
    train_idx = perm[:n_train]
    val_idx   = perm[n_train:n_train+n_val]
    test_idx  = perm[n_train+n_val:]
    SPLIT_FILE.parent.mkdir(parents=True, exist_ok=True)
    with open(SPLIT_FILE, "w") as f:
        json.dump({"train": train_idx, "val": val_idx, "test": test_idx}, f, indent=2)

train_ds = Subset(dataset, train_idx)
val_ds   = Subset(dataset, val_idx)
test_ds  = Subset(dataset, test_idx)

print("Split sizes:", len(train_ds), len(val_ds), len(test_ds))

Split sizes: 1459 182 183


In [119]:
class Wav2Vec2Collator:
    """Prepare raw waveforms for Wav2Vec2 (padding + attention mask + optional aug)."""
    def __init__(self, processor, sr=16000, max_seconds=8, augment=False):
        self.processor = processor
        self.sr = sr
        self.max_len = int(max_seconds * sr)
        self.augment = augment and HAVE_TA

    def _augment(self, x: torch.Tensor) -> torch.Tensor:
        if not self.augment: return x
        # Gaussian noise
        if random.random() < 0.5:
            noise_std = 0.005 * (x.abs().mean().item() + 1e-6)
            x = x + torch.randn_like(x) * noise_std
        # Random gain
        if random.random() < 0.3:
            gain_db = random.uniform(-3.0, 3.0)
            x = x * (10.0 ** (gain_db / 20.0))
        return x

    def __call__(self, batch):
        waves, labels = [], []
        for b in batch:
            w = b["audio"]
            if isinstance(w, np.ndarray):
                w = torch.from_numpy(w)
            w = w.float().view(-1)

            # Crop long clips
            if len(w) > self.max_len:
                start = random.randint(0, len(w) - self.max_len)
                w = w[start:start + self.max_len]

            # Peak normalize
            peak = w.abs().max()
            if peak > 0:
                w = w / peak

            # Light augmentation
            w = self._augment(w)

            waves.append(w.numpy())
            labels.append(int(b["label"]))

        inputs = processor(
            waves,
            sampling_rate=self.sr,
            return_tensors="pt",
            padding=True,
            truncation=False
        )
        inputs["labels"] = torch.tensor(labels, dtype=torch.long)
        return inputs

In [120]:
processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=n_classes,
    ignore_mismatched_sizes=True,
    use_safetensors=True
)

# Label maps in config
model.config.id2label = {i: name for i, name in enumerate(label_names)}
model.config.label2id = {name: i for i, name in enumerate(label_names)}

# ลบส่วน freeze ออก หรือใช้ partial freeze
# if FREEZE_BASE:
#     model.freeze_feature_encoder()

# แทนที่ด้วย: Partial freeze (ถ้าต้องการ)
if FREEZE_BASE:
    # Freeze เฉพาะส่วน feature extractor แต่ปล่อย projection และ classifier
    for name, param in model.named_parameters():
        if 'feature_extractor' in name:
            param.requires_grad = False
        else:
            param.requires_grad = True

model.to(device)

# เพิ่มการตรวจสอบ parameters
print("=== Model Parameters Status ===")
trainable_params = 0
frozen_params = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        trainable_params += param.numel()
        print(f"✓ TRAINABLE: {name} - {param.shape}")
    else:
        frozen_params += param.numel()
        print(f"✗ FROZEN: {name} - {param.shape}")

print(f"\nTrainable parameters: {trainable_params:,}")
print(f"Frozen parameters: {frozen_params:,}")
print(f"Total parameters: {trainable_params + frozen_params:,}")

c:\V89\.venv\Lib\site-packages\transformers\configuration_utils.py:334: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== Model Parameters Status ===
✓ TRAINABLE: wav2vec2.masked_spec_embed - torch.Size([768])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.0.conv.weight - torch.Size([512, 1, 10])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight - torch.Size([512])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias - torch.Size([512])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.1.conv.weight - torch.Size([512, 512, 3])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.2.conv.weight - torch.Size([512, 512, 3])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.3.conv.weight - torch.Size([512, 512, 3])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.4.conv.weight - torch.Size([512, 512, 3])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.5.conv.weight - torch.Size([512, 512, 2])
✓ TRAINABLE: wav2vec2.feature_extractor.conv_layers.6.conv.weight - torch.Size([512, 512, 2])
✓ TRAINABLE: wav2vec2.feature_projection.layer_norm.weight - torch.Si

In [121]:
collate_fn = Wav2Vec2Collator(processor, sr=SAMPLE_RATE, max_seconds=MAX_SECONDS, augment=AUGMENT)

# For notebooks on Windows, keep num_workers=0 (pyedflib may not be multiprocess-safe in all setups)
NUM_WORKERS = 0
PIN_MEMORY = (device.type == "cuda")

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate_fn, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          collate_fn=collate_fn, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          collate_fn=collate_fn, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)



In [122]:
def evaluate_model(model, loader, label_encoder, device, return_arrays=False):
    model.eval()
    y_true, y_pred, y_proba_chunks = [], [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(
                input_values=batch["input_values"],
                attention_mask=batch.get("attention_mask", None)
            ).logits
            proba = torch.softmax(logits, dim=-1).cpu().numpy()
            y_proba_chunks.append(proba)
            y_pred.extend(np.argmax(proba, axis=1))
            y_true.extend(batch["labels"].cpu().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_proba = np.vstack(y_proba_chunks) if len(y_proba_chunks) > 0 else np.zeros((0, len(label_encoder.classes_)))

    class_names = list(label_encoder.classes_)
    n_classes = len(class_names)
    
    # Fix: Use only labels that appear in predictions
    unique_labels = np.unique(np.concatenate([y_true, y_pred]))
    cm = confusion_matrix(y_true, y_pred, labels=unique_labels)

    acc = accuracy_score(y_true, y_pred) if len(y_true) else np.nan
    f1_macro = f1_score(y_true, y_pred, average="macro", zero_division=0) if len(y_true) else np.nan
    f1_weighted = f1_score(y_true, y_pred, average="weighted", zero_division=0) if len(y_true) else np.nan
    prec_macro, rec_macro, _, _ = precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0) if len(y_true) else (np.nan, np.nan, None, None)

    cm_float = cm.astype(float) if cm.size else np.zeros((len(unique_labels), len(unique_labels)), float)
    row_sums = cm_float.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1.0
    bal_acc = (cm_float / row_sums).diagonal().mean() if cm.size else np.nan

    try:
        roc_auc_macro = roc_auc_score(y_true, y_proba, multi_class="ovr", average="macro")
    except Exception:
        roc_auc_macro = np.nan
    try:
        pr_auc_macro = average_precision_score(np.eye(n_classes)[y_true], y_proba, average="macro")
    except Exception:
        pr_auc_macro = np.nan
    try:
        ll = log_loss(y_true, y_proba, labels=list(range(n_classes)))
    except Exception:
        ll = np.nan

    # Fix: Use target_names that match actual labels in predictions
    report_dict = classification_report(
        y_true, y_pred, 
        labels=unique_labels,
        target_names=[class_names[i] for i in unique_labels],
        output_dict=True, zero_division=0
    ) if len(y_true) else {}

    metrics = {
        "accuracy": acc,
        "balanced_accuracy": bal_acc,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "precision_macro": prec_macro,
        "recall_macro": rec_macro,
        "roc_auc_macro": roc_auc_macro,
        "pr_auc_macro": pr_auc_macro,
        "log_loss": ll,
        "support": int(len(y_true))
    }
    out = (metrics, report_dict, cm, class_names)
    if return_arrays:
        return out + (y_true, y_pred, y_proba)
    return out

def save_confusion_matrix(cm, class_names, save_path, normalize=True, title=None):
    cm_plot = cm.astype(float)
    if normalize and cm_plot.size:
        row_sums = cm_plot.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1.0
        cm_plot = cm_plot / row_sums

    plt.figure(figsize=(6, 5))
    plt.imshow(cm_plot, interpolation='nearest')
    plt.title(title or "Confusion Matrix")
    plt.xlabel("Predicted"); plt.ylabel("True")
    ticks = np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right"); plt.yticks(ticks, class_names)

    fmt = ".2f" if normalize else "d"
    thresh = cm_plot.max() / 2. if cm_plot.size else 0.5
    for i in range(cm_plot.shape[0]):
        for j in range(cm_plot.shape[1]):
            val = cm_plot[i, j] if normalize else int(cm[i, j])
            plt.text(j, i, format(val, fmt),
                     ha="center", va="center",
                     color="white" if (cm_plot[i, j] if cm_plot.size else 0) > thresh else "black")
    plt.tight_layout()
    Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(save_path, bbox_inches="tight"); plt.close()

def save_classification_report(report_dict, save_csv_path):
    if not report_dict:
        return None
    df = pd.DataFrame(report_dict).T
    Path(save_csv_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(save_csv_path, index=True)
    return df

def append_metrics_row(results_csv, model_name, split_name, metrics, extras=None):
    row = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "model": model_name,
        "split": split_name,
        **metrics
    }
    if extras:
        row.update(extras)
    if os.path.exists(results_csv):
        df = pd.read_csv(results_csv)
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    else:
        df = pd.DataFrame([row])
    Path(results_csv).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(results_csv, index=False)
    return df.tail(1)

### Train Loop

In [123]:
logits = model(
    input_values=batch["input_values"],
    attention_mask=batch.get("attention_mask", None)
).logits

In [124]:
# แก้ไขเซลล์ #VSC-9cc5bbe8
# กำหนดตัวแปรที่จำเป็น
SPLIT_FILE = Path("./splits/data_split.json")
OUT_DIR = Path(f"./eval_out/{MODEL_TAG}")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ❌ ลบส่วนนี้ออก - เพราะ optimizer ถูกสร้างใหม่หลังจาก model setup
# total_steps  = len(train_loader) * EPOCHS
# warmup_steps = int(total_steps * WARMUP_RATIO)
# optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
# scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

use_amp = (device.type == "cuda")
scaler  = torch.cuda.amp.GradScaler(enabled=use_amp)

# Optional: class weights
if USE_CLASS_WEIGHTS:
    all_y = dataset.label_encoder.transform(dataset.df['type'])
    from sklearn.utils.class_weight import compute_class_weight
    classes = np.arange(n_classes)
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=all_y)
    class_weights = torch.tensor(weights, dtype=torch.float32, device=device)
    ce_loss = nn.CrossEntropyLoss(weight=class_weights)
else:
    ce_loss = nn.CrossEntropyLoss()

# ✅ เพิ่ม optimizer ใหม่หลังจาก model ถูก setup แล้ว
total_steps  = len(train_loader) * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

print("✅ Optimizer created with current model parameters")
print(f"Optimizer parameter groups: {len(optimizer.param_groups)}")
print(f"Parameters in optimizer: {sum(len(group['params']) for group in optimizer.param_groups)}")

C:\Users\Abcde\AppData\Local\Temp\ipykernel_12772\2653787837.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler  = torch.cuda.amp.GradScaler(enabled=use_amp)


✅ Optimizer created with current model parameters
Optimizer parameter groups: 1
Parameters in optimizer: 215


In [126]:
# แทนที่เซลล์ที่เลือกด้วยโค้ดนี้

total_steps  = len(train_loader) * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

use_amp = (device.type == "cuda")
scaler  = torch.cuda.amp.GradScaler(enabled=use_amp)

# Optional: class weights
if USE_CLASS_WEIGHTS:
    all_y = dataset.label_encoder.transform(dataset.df['type'])
    from sklearn.utils.class_weight import compute_class_weight
    classes = np.arange(n_classes)
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=all_y)
    class_weights = torch.tensor(weights, dtype=torch.float32, device=device)
    ce_loss = nn.CrossEntropyLoss(weight=class_weights)
else:
    ce_loss = nn.CrossEntropyLoss()

# เพิ่มเซลล์ใหม่สำหรับ training loop
def run_eval_and_log(split_name, loader):
    metrics, report, cm, class_names = evaluate_model(model, loader, dataset.label_encoder, device)
    print(f"{split_name.upper()}:", metrics)
    save_confusion_matrix(cm, class_names, OUT_DIR / f"cm_{split_name}.png", normalize=True, title=f"{split_name.title()} CM (norm)")
    save_classification_report(report, OUT_DIR / f"cls_report_{split_name}.csv")
    append_metrics_row("./eval_out/scoreboard.csv", MODEL_TAG, split_name, metrics, extras={f"n_{split_name}": len(loader.dataset) if hasattr(loader, "dataset") else None})
    return metrics

for epoch in range(1, EPOCHS + 1):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    running = 0.0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad(set_to_none=True)

        if use_amp:
            with torch.amp.autocast('cuda'):
                outputs = model(
                    input_values=batch["input_values"],
                    attention_mask=batch.get("attention_mask", None)
                )
                logits = outputs.logits
                loss = ce_loss(logits, batch["labels"])
            
            # ✅ เช็คว่า loss เป็น nan หรือไม่
            if torch.isnan(loss) or torch.isinf(loss):
                print(f"Skipping batch due to invalid loss: {loss.item()}")
                continue
                
            scaler.scale(loss).backward()
            
            # ✅ ตรวจสอบ gradients อย่างละเอียด
            total_norm = 0
            valid_grads = 0
            for name, p in model.named_parameters():
                if p.grad is not None:
                    param_norm = p.grad.data.norm(2)
                    if torch.isfinite(param_norm):
                        total_norm += param_norm ** 2
                        valid_grads += 1
                    else:
                        print(f"Invalid gradient in {name}")
            
            total_norm = total_norm ** 0.5
            
            if valid_grads > 0 and torch.isfinite(total_norm) and total_norm > 0:
                # ✅ ลด gradient clipping threshold
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
                scaler.step(optimizer)
                scaler.update()
            else:
                print(f"Skipping optimization step: valid_grads={valid_grads}, grad_norm={total_norm}")
                scaler.update()  # ยังคงต้อง update scaler
        else:
            outputs = model(
                input_values=batch["input_values"],
                attention_mask=batch.get("attention_mask", None)
            )
            logits = outputs.logits
            loss = ce_loss(logits, batch["labels"])
            
            # ✅ เช็คว่า loss เป็น nan หรือไม่
            if torch.isnan(loss) or torch.isinf(loss):
                print(f"Skipping batch due to invalid loss: {loss.item()}")
                continue
                
            loss.backward()
            
            # ✅ ตรวจสอบ gradients อย่างละเอียด
            total_norm = 0
            valid_grads = 0
            for p in model.parameters():
                if p.grad is not None:
                    param_norm = p.grad.data.norm(2)
                    if torch.isfinite(param_norm):
                        total_norm += param_norm ** 2
                        valid_grads += 1
            
            total_norm = total_norm ** 0.5
            
            if valid_grads > 0 and torch.isfinite(total_norm) and total_norm > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
                optimizer.step()
            else:
                print(f"Skipping optimization step: valid_grads={valid_grads}, grad_norm={total_norm}")

        scheduler.step()
        
        # ✅ ป้องกัน nan ใน running loss
        if torch.isfinite(loss):
            running += loss.item()
        
        pbar.set_postfix(
            loss=running / max(1, pbar.n), 
            grad_norm=f"{total_norm:.4f}" if torch.isfinite(total_norm) else "inf"
        )

    metrics = run_eval_and_log("val", val_loader)
    print(f"Epoch {epoch} - Val Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1_macro']:.4f}")

print("Final evaluation on TEST split:")
_ = run_eval_and_log("test", test_loader)

# Save raw preds for analysis
_, _, _, _, y_true_test, y_pred_test, y_proba_test = evaluate_model(
    model, test_loader, dataset.label_encoder, device, return_arrays=True
)
np.save(OUT_DIR / "y_true_test.npy", y_true_test)
np.save(OUT_DIR / "y_pred_test.npy", y_pred_test)
np.save(OUT_DIR / "y_proba_test.npy", y_proba_test)
with open(OUT_DIR / "class_names.json", "w") as f:
    json.dump(label_names, f, ensure_ascii=False, indent=2)

# Save checkpoint
model.save_pretrained("./osa_wav2vec2_ckpt")
processor.save_pretrained("./osa_wav2vec2_ckpt")
print("Done. Artifacts saved to:", OUT_DIR.as_posix())

C:\Users\Abcde\AppData\Local\Temp\ipykernel_12772\2766179936.py:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler  = torch.cuda.amp.GradScaler(enabled=use_amp)
Epoch 1/5:   0%|          | 0/365 [00:00<?, ?it/s]c:\V89\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Epoch 1/5:   0%|          | 1/365 [00:01<07:04,  1.16s/it, grad_norm=212015.7188, loss=1.61]

Invalid gradient in wav2vec2.feature_extractor.conv_layers.0.conv.weight


Epoch 1/5:   1%|          | 2/365 [00:02<05:56,  1.02it/s, grad_norm=159076.9062, loss=3.22]

Invalid gradient in wav2vec2.feature_extractor.conv_layers.0.conv.weight


Epoch 1/5:   1%|          | 3/365 [00:02<05:35,  1.08it/s, grad_norm=60327.4648, loss=2.41] 

...

VAL: {'accuracy': 0.32967032967032966, 'balanced_accuracy': np.float64(0.25), 'f1_macro': 0.12396694214876033, 'f1_weighted': 0.16347289074561802, 'precision_macro': 0.08241758241758242, 'recall_macro': 0.25, 'roc_auc_macro': nan, 'pr_auc_macro': 0.2268638454821721, 'log_loss': 1.209533402563697, 'support': 182}
Epoch 3 - Val Accuracy: 0.3297, F1: 0.1240


Epoch 4/5: 100%|██████████| 365/365 [05:16<00:00,  1.15it/s, grad_norm=4151.5908, loss=1.43] 
c:\V89\.venv\Lib\site-packages\sklearn\metrics\_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


VAL: {'accuracy': 0.43956043956043955, 'balanced_accuracy': np.float64(0.24806397306397307), 'f1_macro': 0.21454632324197542, 'f1_weighted': 0.36182594281017605, 'precision_macro': 0.23783875482420797, 'recall_macro': 0.24806397306397307, 'roc_auc_macro': nan, 'pr_auc_macro': 0.21309067931188178, 'log_loss': 1.1992114003940368, 'support': 182}
Epoch 4 - Val Accuracy: 0.4396, F1: 0.2145


Epoch 5/5: 100%|██████████| 365/365 [05:56<00:00,  1.02it/s, grad_norm=9714.7471, loss=1.42] 
c:\V89\.venv\Lib\site-packages\sklearn\metrics\_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


VAL: {'accuracy': 0.3791208791208791, 'balanced_accuracy': np.float64(0.27605218855218855), 'f1_macro': 0.24962330487192363, 'f1_weighted': 0.3521506118191201, 'precision_macro': 0.27989790957705396, 'recall_macro': 0.27605218855218855, 'roc_auc_macro': nan, 'pr_auc_macro': 0.25580397881009, 'log_loss': 1.1907396579370555, 'support': 182}
Epoch 5 - Val Accuracy: 0.3791, F1: 0.2496
Final evaluation on TEST split:
TEST: {'accuracy': 0.33879781420765026, 'balanced_accuracy': np.float64(0.2022127659574468), 'f1_macro': 0.19676460516402988, 'f1_weighted': 0.3407357007885155, 'precision_macro': 0.22857142857142856, 'recall_macro': 0.2022127659574468, 'roc_auc_macro': 0.5468200331791401, 'pr_auc_macro': 0.23682813281293966, 'log_loss': 1.21849509574613, 'support': 183}
Done. Artifacts saved to: eval_out/wav2vec2-base-osa-win4060-v2


### Evaluate

In [136]:
# Evaluate ทุก split
print("=== EVALUATION RESULTS ===")

# Validation
print("\n📊 VALIDATION SET:")
val_metrics, val_report, val_cm, class_names = evaluate_model(
    model, val_loader, dataset.label_encoder, device
)
print(f"Accuracy: {val_metrics['accuracy']:.4f}")
print(f"F1-Macro: {val_metrics['f1_macro']:.4f}")
print(f"F1-Weighted: {val_metrics['f1_weighted']:.4f}")

# Test
print("\n🎯 TEST SET:")
test_metrics, test_report, test_cm, _ = evaluate_model(
    model, test_loader, dataset.label_encoder, device
)
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"F1-Macro: {test_metrics['f1_macro']:.4f}")
print(f"F1-Weighted: {test_metrics['f1_weighted']:.4f}")

# Save visualizations
save_confusion_matrix(val_cm, class_names, "./eval_results/cm_val.png", 
                     normalize=True, title="Validation Confusion Matrix")
save_confusion_matrix(test_cm, class_names, "./eval_results/cm_test.png", 
                     normalize=True, title="Test Confusion Matrix")

=== EVALUATION RESULTS ===

📊 VALIDATION SET:


c:\V89\.venv\Lib\site-packages\sklearn\metrics\_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


Accuracy: 0.3407
F1-Macro: 0.1956
F1-Weighted: 0.3133

🎯 TEST SET:
Accuracy: 0.3333
F1-Macro: 0.1820
F1-Weighted: 0.3267
